In [4]:
%matplotlib notebook

import openalea.phenomenal.display.notebook as phm_display_notebook
from segmentation_maize import segmentation, plant_seg
import datacloud as dc

In [5]:
db = dc.read_index()

In [6]:
plant_ids = db.plant.unique()
plant_id = plant_ids[2]

In [7]:
plants = db.loc[db.plant==plant_id, :]
plants.sort_values('date', inplace=True)

/var/lib/miniconda3/envs/phenomenal/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
import multiprocessing as mp

In [9]:
num_jobs=mp.cpu_count()
pool = mp.Pool(num_jobs)

/var/lib/miniconda3/envs/phenomenal/lib/python2.7/site-packages/openalea/phenomenal/segmentation/maize_analysis.py:115: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  p_all = numpy.linalg.lstsq(XX, width[::-1])[0]
/var/lib/miniconda3/envs/phenomenal/lib/python2.7/site-packages/openalea/phenomenal/segmentation/maize_analysis.py:115: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  p_all = numpy.linalg.lstsq(XX, width[::-1])[0]
/var/lib/miniconda3/envs/phenomenal/lib/python2.7/site-packages/openalea/phenomenal/segmentation/m

In [10]:
rows = [row for i, row in plants.iterrows()]

In [11]:
vmsis = pool.map(func=plant_seg, iterable=rows)

In [12]:
vmsis= filter(None, vmsis)

In [110]:
from openalea.phenomenal.display.notebook import *
import ipyvolume
def show_segmentation(voxel_segmentation,
                      size=1.0,
                      width=500, height=500):
    ipyvolume.figure(width=width, height=height)
    ipyvolume.view(distance=4., azimuth=90.,elevation=-90)

    def get_color(label, info):

        if label == "stem":
            color = (128, 128, 128)
        elif label == "unknown":
            color = (255, 255, 255)
        elif 'pm_leaf_number' in info:
            color_map = order_color_map()
            color = color_map[info['pm_leaf_number']]
            color = tuple([int(255 * x) for x in color])
        else:
            if label == "growing_leaf":
                color = (255, 0, 0)
            else:
                color = (0, 255, 0)

        return "rgb" + str(color)

    for vo in voxel_segmentation.voxel_organs:

        voxels_position = numpy.array(
            map(tuple, list(vo.voxels_position())))

        plot_voxel(voxels_position,
                   size=size * 1,
                   color=get_color(vo.label, vo.info))

        if ((vo.label == "mature_leaf" or vo.label == "growing_leaf") and
                len(vo.voxel_segments) > 0 and "pm_position_tip" in vo.info):

            plot_voxel(numpy.array([vo.info['pm_position_tip']]),
                       size=size * 2,
                       color="red",
                       marker="sphere")

            plot_voxel(numpy.array([vo.info['pm_position_base']]),
                       size=size * 2,
                       color="blue",
                       marker="sphere")

    voxels_position = numpy.array(list(
        voxel_segmentation.get_voxels_position()))

    x_min = voxels_position[:, 0].min()
    x_max = voxels_position[:, 0].max()
    y_min = voxels_position[:, 1].min()
    y_max = voxels_position[:, 1].max()
    z_min = voxels_position[:, 2].min()
    z_max = voxels_position[:, 2].max()
    xyz_max = max(x_max - x_min, y_max - y_min, z_max - z_min)
    ipyvolume.xlim(x_min, x_min + xyz_max)
    ipyvolume.ylim(y_min, y_min + xyz_max)
    ipyvolume.zlim(z_min, z_min + xyz_max)
    ipyvolume.show()

In [120]:
for i, vmsi in enumerate(vmsis):
    fname='plant_%s_%d.json.gz'
    row= rows[i]
    vmsi.write_to_json_gz(fname%(row.genotype, row.timestamp))

In [116]:
row.timestamp


1466511116